In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
xtrain_df = pd.read_csv('task/train_set/x_train.csv')
xtrain_agg_df = pd.read_csv('task/train_set/x_train_aggregated.csv')
ytrain_df = pd.read_csv('task/train_set/y_train.csv')
xval_agg_df = pd.read_csv('task/validation_set/x_val_aggregated.csv')
yval_df = pd.read_csv('task/validation_set/y_val.csv')

In [3]:
xtrain_df.head()

,Hour,Action,Amount,AccountID,External,OldBalance,NewBalance,isUnauthorizedOverdraft
0,1,CASH_IN,133687.16,C9618460261,NaN,104703.85,238391.02,0
1,1,CASH_IN,171878.50,C0742280337,NaN,193747.95,365626.45,0
2,1,CASH_IN,110503.91,C1337418745,NaN,6.40,110510.31,0
3,1,CASH_IN,158292.33,C7477491662,NaN,87.03,158379.36,0
4,1,CASH_IN,208342.54,C7477491662,NaN,158379.36,366721.91,0


In [4]:
xtrain_df.describe()

,Hour,Amount,OldBalance,NewBalance,isUnauthorizedOverdraft
count,1.985971e+06,1.985971e+06,1.985971e+06,1.985971e+06,1.985971e+06
mean,1.850364e+02,1.379793e+05,3.047470e+06,3.037178e+06,3.496980e-02
std,1.256821e+02,3.026488e+05,2.049241e+06,2.014591e+06,1.837034e-01
min,1.000000e+00,2.000000e-02,-1.990282e+05,-1.990282e+05,0.000000e+00
25%,4.500000e+01,1.217673e+04,1.897720e+06,1.926878e+06,0.000000e+00
50%,1.870000e+02,9.046182e+04,3.219526e+06,3.211457e+06,0.000000e+00
75%,2.810000e+02,1.667487e+05,4.048426e+06,4.026252e+06,0.000000e+00
max,7.190000e+02,1.814506e+07,3.020602e+07,3.003953e+07,1.000000e+00


In [5]:
xtrain_agg_df.head()

,AccountID,NumTransactions,AvgAmount,MaxAmount,TotalIn,MaxIn,TotalOut,MaxCashOut,MaxDebit,MaxTransfer,MaxPayment,MaxFreqH,MaxFreqD
0,C0001143040,562,146658.532989,3585064.01,35124031.74,424498.32,47298063.80,284531.00,8893.26,3585064.01,27070.99,12,84
1,C0001332727,178,128888.997753,903276.53,13396604.25,341238.53,9545637.35,258056.33,0.00,903276.53,21396.14,6,33
2,C0002641069,56,146342.356786,2571512.49,4547603.60,293992.40,3647568.38,204095.40,3594.16,2571512.49,14710.21,2,8
3,C0002967890,103,117303.432330,3554689.54,3016367.57,392633.82,9065885.96,247445.60,23100.20,3554689.54,24640.16,4,22
4,C0003513711,50,144333.040800,1563406.66,4462829.33,264727.14,2753822.71,142728.98,3080.34,1563406.66,16098.56,2,8


In [6]:
xtrain_agg_df.describe()

,NumTransactions,AvgAmount,MaxAmount,TotalIn,MaxIn,TotalOut,MaxCashOut,MaxDebit,MaxTransfer,MaxPayment,MaxFreqH,MaxFreqD
count,11061.000000,1.106100e+04,1.106100e+04,1.106100e+04,1.106100e+04,1.106100e+04,1.106100e+04,11061.000000,1.106100e+04,11061.000000,11061.000000,11061.000000
mean,173.268421,1.454351e+05,1.866861e+06,1.042735e+07,3.393609e+05,1.227517e+07,2.308254e+05,5364.761463,1.815955e+06,19479.734623,5.223488,28.436037
std,1122.199246,6.548711e+04,1.318804e+06,2.292905e+07,5.875693e+05,2.354127e+07,1.153758e+05,7669.151693,1.290086e+06,7948.617402,17.342272,176.490148
min,1.000000,4.407557e+03,8.437110e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,1.000000,1.000000
25%,39.000000,1.196196e+05,8.641564e+05,2.128583e+06,2.599915e+05,2.558064e+06,1.981320e+05,0.000000,8.604498e+05,15773.650000,2.000000,7.000000
50%,99.000000,1.387191e+05,1.780288e+06,6.165977e+06,3.080791e+05,8.088935e+06,2.453406e+05,3058.480000,1.776725e+06,20428.490000,4.000000,17.000000
75%,215.000000,1.561108e+05,2.625486e+06,1.351976e+07,3.534323e+05,1.671086e+07,2.800967e+05,7951.330000,2.616498e+06,24225.720000,6.000000,35.000000
max,77542.000000,1.804926e+06,1.705500e+07,1.392348e+09,1.705500e+07,1.400163e+09,4.008213e+06,162019.560000,1.206550e+07,78382.870000,1206.000000,12265.000000


In [7]:
ytrain_df.describe()

,Fraudster
count,11061.000000
mean,0.157129
std,0.363938
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [8]:
xtrain_df.dtypes

Hour                         int64
Action                      object
Amount                     float64
AccountID                   object
External                    object
OldBalance                 float64
NewBalance                 float64
isUnauthorizedOverdraft      int64
dtype: object

In [9]:
xtrain_df.isnull().sum()

Hour                             0
Action                           0
Amount                           0
AccountID                        0
External                   1269776
OldBalance                       0
NewBalance                       0
isUnauthorizedOverdraft          0
dtype: int64

In [10]:
xtrain_agg_df.dtypes

AccountID           object
NumTransactions      int64
AvgAmount          float64
MaxAmount          float64
TotalIn            float64
MaxIn              float64
TotalOut           float64
MaxCashOut         float64
MaxDebit           float64
MaxTransfer        float64
MaxPayment         float64
MaxFreqH             int64
MaxFreqD             int64
dtype: object

In [11]:
fraudster_counts = yval_df['Fraudster'].value_counts()
fraudster_counts_sorted = fraudster_counts.sort_values(ascending=False)
print(fraudster_counts_sorted)


Fraudster
0    9408
1    1652
Name: count, dtype: int64


In [12]:
x = xtrain_agg_df[['NumTransactions','AvgAmount','MaxAmount','TotalIn','MaxIn','TotalOut','MaxCashOut','MaxDebit','MaxTransfer','MaxPayment','MaxFreqH','MaxFreqD']]
y = ytrain_df['Fraudster']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [13]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(x, y)
#y_pred = rf_model.predict(x_test)
# Evaluation
#accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

RandomForestClassifier(random_state=42)

In [14]:
xval = xval_agg_df[['NumTransactions','AvgAmount','MaxAmount','TotalIn','MaxIn','TotalOut','MaxCashOut','MaxDebit','MaxTransfer','MaxPayment','MaxFreqH','MaxFreqD']]
yval = yval_df['Fraudster']
y_pred = rf_model.predict(xval)
# Evaluation
accuracy = accuracy_score(yval, y_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))
print(classification_report(yval, y_pred))

Accuracy: 0.8500904159132007
[[9400    8]
 [1650    2]]
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      9408
           1       0.20      0.00      0.00      1652

    accuracy                           0.85     11060
   macro avg       0.53      0.50      0.46     11060
weighted avg       0.75      0.85      0.78     11060

